In [30]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from utils import clean_target
from categorical_ordinal import get_categorical_ordinal_columns
from categorical_nominal import get_categorical_nominal_columns
from columns_transformers import ColumnSelector
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

<font color="orange"> <b>Grupos de features:</b></font>

- **Categorial Ordinal:**
    - TP_ (17-4-3 = 10)
    - Questions: ```["Q001", "Q002", "Q003", "Q004", "Q005","Q006", "Q007", "Q008", "Q009", "Q010", "Q011", "Q012", "Q013", "Q014", "Q015", "Q016", "Q017", "Q019", "Q022", "Q024"]``` (20)
                     
- **Categorial Nominal:**
    - IN_ : All Binary (52) 
    - TP_ : ```["TP_SEXO", "TP_ESTADO_CIVIL", "TP_COR_RACA", "TP_NACIONALIDADE"]``` (4)
    - SG_ : (4-1 = 3)
    - Questions: ```["Q018", "Q020", "Q021", "Q023", "Q025"]``` (5)
    
- **Numerical:**
    - NU_IDADE (1)

- Droped:
    - Identificator: ```[NU_INSCRICAO]``` (1)
    - More than 40% missing: ```['CO_ESCOLA', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC']``` (4)
    - NO_M: (To many categories): ```['NO_MUNICIPIO_RESIDENCIA', 'NO_MUNICIPIO_NASCIMENTO', 'NO_MUNICIPIO_PROVA']``` (3)
    - NU_NOTA: Targets variables (5)

In [31]:
train_df = pd.read_parquet("data/train.parquet")
clean_target(train_df)
#test= pd.read_parquet("data/test.parquet")

In [32]:
categorical_ordinal_columns = get_categorical_ordinal_columns(train_df)
qtd_categorical_ordinal_columns=len(categorical_ordinal_columns)
print(f"Number of categorial ordinal features: {qtd_categorical_ordinal_columns}")

Number of categorial ordinal features: 30


In [33]:
categorical_nominal_columns = get_categorical_nominal_columns(train_df)
qtd_categorical_nominal_columns = len(categorical_nominal_columns)
print(f"Number of categorial nominal features: {qtd_categorical_nominal_columns}")

Number of categorial nominal features: 64


In [34]:
drop_columns = ["NU_INSCRICAO", 
                "CO_ESCOLA", 
                "NO_MUNICIPIO_ESC", 
                "SG_UF_ESC", 
                "TP_DEPENDENCIA_ADM_ESC", 
                "TP_LOCALIZACAO_ESC", 
                "TP_SIT_FUNC_ESC", 
                "NO_MUNICIPIO_RESIDENCIA", 
                "NO_MUNICIPIO_NASCIMENTO", 
                "NO_MUNICIPIO_PROVA"]
qtd_drop_columns = len(drop_columns)
print(f"Number of columns dropped: {qtd_drop_columns}")

Number of columns dropped: 10


In [35]:
target_columns = train_df.filter(regex="NU_NOTA").columns.tolist()
qtd_target_columns = len(target_columns)
print(f"Number of targets: {qtd_target_columns}")

Number of targets: 5


In [36]:
numerical_columns = ["NU_IDADE"]
qtd_numerical_columns = len(numerical_columns)
print(f"Number of targets: {qtd_numerical_columns}")

Number of targets: 1


In [37]:
target_columns = train_df.filter(regex="NU_NOTA").columns.tolist()
qtd_target_columns = len(target_columns)
print(f"Number of targets: {qtd_target_columns}")

Number of targets: 5


In [38]:
all_columns = drop_columns + categorical_nominal_columns + categorical_ordinal_columns + numerical_columns + target_columns
qtd_total = qtd_drop_columns + qtd_categorical_nominal_columns + qtd_categorical_ordinal_columns + qtd_numerical_columns + qtd_target_columns
print(f"Total columns: {qtd_total}")

Total columns: 110


## **Create Pipeline**

In [39]:
"""
Variáveis categóricas com dados ordinais que tem dados faltantes:
- TP_ENSINO: Suposto que NaN representa a categoria faltante descrita nos metadados.
- TP_STATUS_REDACAO: Mapeado para outra classe (Faltou na prova)
"""
categorical_ordinal_pipe = Pipeline([
    ('selector', ColumnSelector(categorical_ordinal_columns)),
    ('imputer', SimpleImputer(missing_values=np.nan, 
                              strategy='constant', 
                              fill_value=0)),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

"""
Variáveis categóricas com dados ordinais que tem dados faltantes:
- SG_UF_NASCIMENTO: Mapeado para uma nova categoria
"""
categorical_nominal_pipe = Pipeline([
    ('selector', ColumnSelector(categorical_nominal_columns)),
    ('imputer', SimpleImputer(missing_values=np.nan, 
                              strategy='constant', 
                              fill_value="missing")),
    ('encoder', OneHotEncoder(drop="first", handle_unknown='ignore'))
])

numerical_pipe = Pipeline([
    ('selector', ColumnSelector(numerical_columns)),
    ('imputer', SimpleImputer(missing_values=np.nan, 
                              strategy='constant', 
                              fill_value=0)),
    ('scaler', MinMaxScaler())
])


preprocessor = FeatureUnion([
    ('categorical_ordinal', categorical_ordinal_pipe),
    ('categorical_nominal', categorical_nominal_pipe),
    ('numerical', numerical_pipe)
])

kwargs_regressor = {"n_estimators":50,
                    "n_jobs":-1,
                    "verbose":2}

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', VarianceThreshold(threshold=0.05)),
    ('model', RandomForestRegressor(**kwargs_regressor))
])

In [40]:
n_samples = 1000
X = train_df.sample(n_samples).drop(columns=target_columns+drop_columns)
y = train_df.sample(n_samples).filter(regex="NU_NOTA")

In [41]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.1, random_state=42)

In [42]:
def split_target(y):
    y_nu_nota_cn = y["NU_NOTA_CN"]
    y_nu_nota_ch = y["NU_NOTA_CH"]
    y_nu_nota_lc = y["NU_NOTA_LC"]
    y_nu_nota_mt = y["NU_NOTA_MT"]
    y_nu_nota_redacao = y["NU_NOTA_REDACAO"]
    return (y_nu_nota_cn, y_nu_nota_ch, y_nu_nota_lc, y_nu_nota_mt, y_nu_nota_redacao)

y_train_cn, y_train_ch, y_train_lc, y_train_mt, y_train_redacao = split_target(y_train)
y_test_cn, y_test_ch, y_test_lc, y_test_mt, y_test_redacao = split_target(y_test)

y_structure = {"NU_NOTA_CN":[y_train_cn, y_test_cn], 
               "NU_NOTA_CH":[y_train_ch, y_test_ch],
               "NU_NOTA_LC":[y_train_lc, y_test_lc],
               "NU_NOTA_MT":[y_train_mt, y_test_mt],
               "NU_NOTA_REDACAO":[y_train_redacao, y_test_redacao]}

In [43]:
from joblib import dump

for key, ys in tqdm(y_structure.items()):
    
    pipe.fit(X_train, ys[0])
    dump(pipe, f"models/model_{key}.joblib") 
    y_train_hat = pipe.predict(X_train)
    ys.append(y_train_hat)
    y_test_hat = pipe.predict(X_test)
    ys.append(y_test_hat)

  0%|                                                                                                                                                                                    | 0/5 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.1s


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50building tree 14 of 50

building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50building tree 19 of 50
building tree 20 of 50

building tree 21 of 50
building tree 22 of 50building tree 23 of 50

building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50building tree 30 of 50

building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50building tree 38 of 50building tree 39 of 50building tree 40 of 50



building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
 20%|██████████████████████████████████▍                                                                                                                                         | 1/5 [00:00<00:01,  2.52it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.1s


building tree 1 of 50
building tree 2 of 50building tree 3 of 50

building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50building tree 13 of 50

building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50building tree 22 of 50

building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50building tree 29 of 50

building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50building tree 34 of 50

building tree 35 of 50building tree 36 of 50

building tree 37 of 50building tree 38 of 50

building tree 39 of 50
building tree 40 of 50
building tree 41 of 50building tree 42 of 50

building tree 43 of 50
building tree 44 of 

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
 40%|████████████████████████████████████████████████████████████████████▊                                                                                                       | 2/5 [00:00<00:01,  2.56it/s]

...

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                    | 3/5 [00:01<00:00,  2.54it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.1s


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50building tree 15 of 50
building tree 16 of 50

building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50building tree 21 of 50
building tree 22 of 50
building tree 23 of 50

building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50building tree 30 of 50

building tree 31 of 50building tree 32 of 50

building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50building tree 39 of 50
building tree 40 of 50

building tree 41 of 50
building tree 42 of 50building tree 43 of 50

building tree 44 of 

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                  | 4/5 [00:01<00:00,  2.49it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.1s


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50building tree 11 of 50
building tree 12 of 50

building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50building tree 19 of 50

building tree 20 of 50building tree 21 of 50

building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50building tree 29 of 50

building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50building tree 43 of 50

building tree 44 of 

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.53it/s]


In [44]:
for key, ys in tqdm(y_structure.items()):
    train_error = mean_squared_error(ys[0], ys[2], squared=False)
    test_error = mean_squared_error(ys[1], ys[3], squared=False)
    print(key)
    print(f"Train: {train_error}")
    print(f"Test: {test_error}\n")    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1737.78it/s]

NU_NOTA_CN
Train: 88.1096430407417
Test: 232.37107216559087

NU_NOTA_CH
Train: 88.05912658935578
Test: 227.5229472546098

NU_NOTA_LC
Train: 90.26104040372509
Test: 233.02540975415135

NU_NOTA_MT
Train: 101.28760922697943
Test: 262.99910421415

NU_NOTA_REDACAO
Train: 114.7048153595422
Test: 308.4948414479568

